#  Enhanced Scoring for Overlapping AMC Audiences Playbook - Code

This code helps you leverage the Persona API, AMC data, and the Audience Management API's "Discovery Post Audience Segments" feature to discover canonical IDs and extract scoring information on audiences that overlap with AMC Audiences.

Before running the following code snippets, please follow these steps:

1. Locate the Configurations.ipynb file included in the zip package you downloaded.

2. Open and run the Configurations.ipynb notebook to set up your environment. This configuration step is crucial as it will enable the necessary settings and dependencies for the subsequent code to function correctly.

Once you've completed the configuration process, you can proceed with the code snippets that leverage the Persona API, AMC data, and the Audience Management API.

In [ ]:
# Configuring the Account & Marketplace
%run ./Configuration.ipynb

## Identify the Profiles for your DSP advertisers

In [ ]:
profiles = get_profiles(Clientid, access_token, url)
profiles_df = pd.DataFrame(profiles)
display(profiles_df.head())

display(profiles)

In [ ]:
# Assigning the DSP & SP profiles 
dsp_profile_id = str(profiles[0]['profileId'])
sp_profile_id = str(profiles[3]['profileId'])

### Retrieve advertisers - Identify the  advertiser to perform the Persona analysis

In [ ]:
headers = {
                'content-type': 'application/json',
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Authorization': 'Bearer ' + access_token
            }

print(f'DSP Profile ID : {dsp_profile_id}\n')
display(headers)

In [ ]:
response = requests.get(url + '/dsp/advertisers/', headers = headers)
json.loads(response.text)

In [ ]:
# Selects the first Advertiser
advertiser_id = str(json.loads(response.text)['response'][0]['advertiserId'])
advertiser_id

### Read the AMC output report (generated as a CSV file) 

In [ ]:
# Read CSV 
# You can update this by using a table in Athena that stores results from AMC API
dataframe_sample = pd.read_csv('data/SAMPLE-DEMO-PERSONAAPI.csv') 

# Drop all the null values in the segment
dataframe_sample.dropna(subset=['segment'], inplace=True)

dataframe_sample.head()


In [ ]:

## Use the below segment names in the Audience discovery API to generate a canonical value
print(dataframe_sample[['segment']].to_string(index=False)) 

### Extract the canonical ID from the internal Segment ID from AMC output

#### Refer to section - Discovery Post Audience Segments to obtain canonical IDs

**Sample Canonical IDs:**

- 369587163451914839
- 401286908546984562
- 415420091813035927
- 373288256866650707
- 379549583597537886
- 402656589620600641
- 416818731248919256
- 412584098030288470
- 400620232007795962
- 372520061094241926
- 416442185032790688
- 412584098030288470
- 390934590405599484
- 419044541308536170
- 409072386332050204
- 400421201299530385
- 364392204283695266
- 369906023970224565
- 387761861532054880
- 400699395028477348

###  Leverage Persona Builder API endpoints with AMC Outputs to analyze affinity and score across: 
- Top categories
- Banded size 
- Overlapping audiences
- Prime video insights 
- Demographics

## Identify the top 5 performing audiences from the advertiser account in AMC

In [9]:
headers = {
                
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Accept': 'application/vnd.topcategoriespurchasedinsights.v1+json',
                'Authorization': 'Bearer ' + access_token
            }

### Retail categories 

In [10]:
# Top Retail categories for the audiences below
body = {
    'audienceTargetingExpression': {
        'audiences': [
            {
                "negative": "false",
                "groupId": "usecase1_retailcategories",
                "audienceId": "404819612694400216"
            },
            {
                "negative": "false",
                "groupId": "personaApiTest",
                "audienceId": "364392204283695266"
            }
        ]
    }
}

query = {'advertiserId': advertiser_id}

### Response extracted is paginated you can use the next token to obtain additional results and sort it by score and affinity

In [ ]:
topcategories_response= json.loads(requests.post(url + '/insights/topCategoriesPurchased', json = body, headers = headers,params = query).text)
display(topcategories_response) 

In [ ]:
##Normalize json

data3=pd.json_normalize(topcategories_response['retailCategories'])
df_categories=pd.DataFrame(data3)
print(df_categories)


df_categories.to_csv("df_Categories_test.csv", index=False)

 ### Banded size

In [13]:
headers = {
                
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Accept': 'application/vnd.bandedsizeinputexpression.v1+json',
                'Authorization': 'Bearer ' + access_token
            }

In [14]:
# Banded Size for the audiences below
body = {
    'audienceTargetingExpression': {
        'audiences': [
            {
                "negative": "false",
                "groupId": "usecase1_bandedsize",
                "audienceId": "404819612694400216"
            },
            {
                "negative": "false",
                "groupId": "personaApiTest",
                "audienceId": "364392204283695266"
            }
        ]
    }
}

query = {'advertiserId': advertiser_id}

In [ ]:
Bandedsize_response= json.loads(requests.post(url + '/insights/bandedSize', json = body, headers = headers,params = query).text)
display(Bandedsize_response) 

In [ ]:
##Normalize json

data3=pd.json_normalize(Bandedsize_response['estimatedSize'])
df_bandedsize=pd.DataFrame(data3)
print(df_bandedsize)
df_categories.to_csv("df_bandedsize_test.csv", index=False)



### Demographics Data

In [17]:
headers = {
                
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Accept': 'application/vnd.demographicinputexpressions.v1+json',
                'Authorization': 'Bearer ' + access_token
            }

In [18]:
# Demographics audiences overlap below
body = {
    'audienceTargetingExpression': {
        'audiences': [
            {
                "negative": "false",
                "groupId": "usecase1_demographics",
                "audienceId": "404819612694400216"
            },
            {
                "negative": "false",
                "groupId": "usecase1_demographicsaud2",
                "audienceId": "364392204283695266"
            }
        ]
    }
}

query = {'advertiserId': advertiser_id}

In [ ]:
demographic_response= json.loads(requests.post(url + '/insights/demographics', json = body, headers = headers,params = query).text)
display(demographic_response) 

In [ ]:
##Normalize json

data3=pd.json_normalize(demographic_response['demographics'])
data=pd.DataFrame(data3)
print(demographic_response['demographics'])

In [ ]:
result=[]

for category, values in data.items():
    for value in values[0]:
        segment = {
            'segmentId': value.get('segmentId'),
            'category': category,
            'attribute': value.get('attribute', value.get('currency',None)),
            'affinity': value['insightMetrics'].get('affinity'),
            'overlapPercentage': value['insightMetrics'].get('overlapPercentage'),
            'Range': str(value.get('range', {}).get('min','') )+ '-' + str(value.get('range', {}).get('max', '')) 
           # 'max': value.get('range', {}).get('max', None)
        }
        
        result.append(segment)
df_demographics = pd.DataFrame(result)
print(df_demographics)

df_demographics.to_csv("df_demographics_test.csv", index=False)

# Obtain Prime Video insights

In [22]:
headers = {
                
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Accept': 'application/vnd.primevideoinputexpressions.v1+json',
                'Authorization': 'Bearer ' + access_token
            }

In [23]:
# Prime video insights
body = {
    'audienceTargetingExpression': {
        'audiences': [
            {
                "negative": "false",
                "groupId": "usecase1_primevideo",
                "audienceId": "404819612694400216"
            },
            {
                "negative": "false",
                "groupId": "usecase1_primevideo2",
                "audienceId": "364392204283695266"
            }
        ]
    }
}

query = {'advertiserId': advertiser_id}

In [ ]:
primevideo_response= json.loads(requests.post(url + '/insights/primeVideo', json = body, headers = headers,params = query).text)
display(primevideo_response) 

In [ ]:
##Normalize json

data3=pd.json_normalize(primevideo_response['primeVideoInsights'])
df_primevideo_response=pd.DataFrame(data3)
print(df_primevideo_response)

In [ ]:
result1= []


for category, values in df_primevideo_response.items():
    #print(type(values[0]))
    if type(values[0]) == list:
        for value in values[0]:
            segment = {
               # 'category': category,
                'actor': value.get('actor'),
                'director': value.get('director'),
                'genre': value.get('genre'),
                'movie': value.get('movie'),
                'series': value.get('series'),
                'affinity': value['insight'].get('affinity'),
                'overlapPercentage': value['insight'].get('overlapPercentage')

                    }
            result1.append(segment)
df_primevideo = pd.DataFrame(result1)
print(df_primevideo)

df_primevideo.to_csv("df_primevideo_test.csv", index=False)

### Overlapped Audiences 

In [27]:
headers = {
                
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Accept': 'application/vnd.topoverlappingaudiencesinputexpression.v1+json',
                'Authorization': 'Bearer ' + access_token
            }

In [28]:
# Overlapped audiences overlap below
body = {
    'audienceTargetingExpression': {
        'audiences': [
            {
                "negative": "false",
                "groupId": "usecase1_overlapped",
                "audienceId": "404819612694400216"
            },
            {
                "negative": "false",
                "groupId": "usecase1_overlapp2",
                "audienceId": "364392204283695266"
            }
        ]
    }
}

query = {'advertiserId': advertiser_id}

In [ ]:
overlappedaudiences_response= json.loads(requests.post(url + '/insights/topOverlappingAudiences', json = body, headers = headers,params = query).text)
display(overlappedaudiences_response) 

In [ ]:
##Normalize json

data3=pd.json_normalize(overlappedaudiences_response['audiences'])
df_overlapaudience_response=pd.DataFrame(data3)
print(df_overlapaudience_response)

df_overlapaudience_response.to_csv("df_overlapaudience_test.csv", index=False)

### Discovery Post Audience Segments - Additional Information on DSP Audiences can be obtained from below

In [ ]:
headers = {
                'content-type': 'application/json',
                'Amazon-Advertising-API-ClientId': Clientid,
                'Amazon-Advertising-API-Scope': dsp_profile_id,
                'Authorization': 'Bearer ' + access_token
            }

In [ ]:
query = {'advertiserId': advertiser_id}
print(query)

LS - Similar to Parents with Children in Household

                                                   Demo - Presence of Children age 4-6
                       2022_Brand_A_Kids_90_Trolls World Tour Conquesting-ProductViews
                                  2022_Brand_A_Toothbrush_90_All ASINs-ProductSearches
                 2022_Brand_A_Toothbrush_30_Adjacent Category Shoppers-ProductSearches
                               2022_Brand_A_Toothbrush_90_Conquesting-ProductPurchases
                                       2022 Brand_A Toothpaste 30 Days-ProductSearches
                                                          IM - Shampoo and Conditioner
                                                          IM - Premium Beauty Products
                                        2022 Brand_A Whitening 30 Days-ProductSearches
                                       2022 Brand_A Whitening 90 Days-ProductPurchases
                                        AMC Repurchases - Cross Brand - Toothbrushes-ProductPurchases

In [ ]:
##body = {'adType': 'DSP',"filters": [{"field": "audienceName","values": ["LS - Similar to Parents with Children in Household","Demo - Presence of Children age 4-6","2022_Brand_A_Kids_90_Trolls World Tour Conquesting-ProductViews","2022_Brand_A_Toothbrush_90_All ASINs-ProductSearches"]}]}

In [ ]:
body = {'adType': 'DSP',"filters": [{"field": "audienceName","values": ["AMC Repurchases - Cross Brand - Toothbrushes-ProductPurchases"]}]}

In [ ]:
response = requests.post(url + '/audiences/list', json = body, headers = headers,params = query)

In [ ]:
data=json.loads(response.text)
print(data)

In [ ]:
#Load Data
data=json.loads(response.text)


# Normalize the Audiences Values
data3=pd.json_normalize(data['audiences'])


# Create a Dataframe for Audiences . AudienceID is the canonical ID for the above selected audiences
df_audiences=pd.DataFrame(data3)
print(df_audiences)

# #Identify the Next Token
# data4=pd.json_normalize(data)
# nextToken=data4['nextToken'][0]

# #Update the query to include the nextToken and re run the request
# query = {'advertiserId': advertiser_id, 'nextToken':nextToken}
# body = {'adType': 'DSP'}
# response = requests.post(base_url + '/audiences/list', json = body, headers = headers,params = query)

# #Updates Value after the nextToken
# data1=json.loads(response.text)

# data_afternextToken=pd.json_normalize(data1['audiences'])

In [ ]:
print(df_audiences[['audienceId']].to_string(index=False)) 

In [ ]:
## You can obtain the audience description to identify which canonical ID you want to choose.

response1 = requests.post(url + '/audiences/list', json = body, headers = headers,params = query)

json.loads(response1.text)